In [1]:
import pandas as pd 

Files = [["DataFiles\JC-202006-citibike-tripdata.csv","DataFiles\\NY-202006-citibike-tripdata.csv"],
         ["DataFiles\JC-202005-citibike-tripdata.csv","DataFiles\\NY-202005-citibike-tripdata.csv"],
         ["DataFiles\JC-202004-citibike-tripdata.csv","DataFiles\\NY-202004-citibike-tripdata.csv"],
         ["DataFiles\JC-202003-citibike-tripdata.csv","DataFiles\\NY-202003-citibike-tripdata.csv"],
         ["DataFiles\JC-202002-citibike-tripdata.csv","DataFiles\\NY-202002-citibike-tripdata.csv"],
         ["DataFiles\JC-202001-citibike-tripdata.csv","DataFiles\\NY-202001-citibike-tripdata.csv"],
         ["DataFiles\JC-201912-citibike-tripdata.csv","DataFiles\\NY-201912-citibike-tripdata.csv"],
         ["DataFiles\JC-201911-citibike-tripdata.csv","DataFiles\\NY-201911-citibike-tripdata.csv"],
         ["DataFiles\JC-201910-citibike-tripdata.csv","DataFiles\\NY-201910-citibike-tripdata.csv"],
         ["DataFiles\JC-201909-citibike-tripdata.csv","DataFiles\\NY-201909-citibike-tripdata.csv"],
         ["DataFiles\JC-201908-citibike-tripdata.csv","DataFiles\\NY-201908-citibike-tripdata.csv"],
         ["DataFiles\JC-201907-citibike-tripdata.csv","DataFiles\\NY-201907-citibike-tripdata.csv"],
         ["DataFiles\JC-201906-citibike-tripdata.csv","DataFiles\\NY-201906-citibike-tripdata.csv"],
         ["DataFiles\JC-201905-citibike-tripdata.csv","DataFiles\\NY-201905-citibike-tripdata.csv"],
         ["DataFiles\JC-201904-citibike-tripdata.csv","DataFiles\\NY-201904-citibike-tripdata.csv"],
         ["DataFiles\JC-201903-citibike-tripdata.csv","DataFiles\\NY-201903-citibike-tripdata.csv"],
         ["DataFiles\JC-201902-citibike-tripdata.csv","DataFiles\\NY-201902-citibike-tripdata.csv"],
         ["DataFiles\JC-201901-citibike-tripdata.csv","DataFiles\\NY-201901-citibike-tripdata.csv"],
         ["DataFiles\JC-201812-citibike-tripdata.csv","DataFiles\\NY-201812-citibike-tripdata.csv"],
         ["DataFiles\JC-201811-citibike-tripdata.csv","DataFiles\\NY-201811-citibike-tripdata.csv"],
         ["DataFiles\JC-201810-citibike-tripdata.csv","DataFiles\\NY-201810-citibike-tripdata.csv"],
         ["DataFiles\JC-201809-citibike-tripdata.csv","DataFiles\\NY-201809-citibike-tripdata.csv"],
         ["DataFiles\JC-201808-citibike-tripdata.csv","DataFiles\\NY-201808-citibike-tripdata.csv"],
         ["DataFiles\JC-201807-citibike-tripdata.csv","DataFiles\\NY-201807-citibike-tripdata.csv"],
         ["DataFiles\JC-201806-citibike-tripdata.csv","DataFiles\\NY-201806-citibike-tripdata.csv"],
         ["DataFiles\JC-201805-citibike-tripdata.csv","DataFiles\\NY-201805-citibike-tripdata.csv"],
         ["DataFiles\JC-201804-citibike-tripdata.csv","DataFiles\\NY-201804-citibike-tripdata.csv"],
         ["DataFiles\JC-201803-citibike-tripdata.csv","DataFiles\\NY-201803-citibike-tripdata.csv"],
         ["DataFiles\JC-201802-citibike-tripdata.csv","DataFiles\\NY-201802-citibike-tripdata.csv"],
         ["DataFiles\JC-201801-citibike-tripdata.csv","DataFiles\\NY-201801-citibike-tripdata.csv"]]


df_TripSummary = pd.DataFrame()

for file in Files:
    print(file)
    dataJC = pd.read_csv(file[0])
    dataNYC = pd.read_csv(file[1])

    # Remove invalid Latitude
    dataJC = dataJC.loc[dataJC["start station latitude"] < 41]
    dataNYC = dataNYC.loc[dataNYC["start station latitude"] < 41]

    # load trip data into dataframes
    df_JCTrips = dataJC[["tripduration","starttime","stoptime","start station id","end station id","bikeid","usertype","birth year"]]

    df_NYCTrips = dataNYC[["tripduration","starttime","stoptime","start station id","end station id","bikeid","usertype","birth year"]]

    df_ALLTrips = df_JCTrips.append(df_NYCTrips)

    #convert date to month, year,day of week
    df_ALLTrips['year'] = pd.DatetimeIndex(df_ALLTrips['starttime']).year
    df_ALLTrips['month'] = pd.DatetimeIndex(df_ALLTrips['starttime']).month
    df_ALLTrips['day_of_week'] = pd.DatetimeIndex(df_ALLTrips['starttime']).day_name()

    # Removed trips more than 1 day
    df_ALLTrips = df_ALLTrips.loc[df_ALLTrips["tripduration"] < 86400]
    
    # get Starting station counts by year, month, day of week
    df_StartStation = df_ALLTrips.groupby(['start station id','year','month','day_of_week']).size().reset_index(name='StartCount')
    df_StartStation = df_StartStation.sort_values(by=["StartCount"])
    df_StartStation.columns = ["station id","year","month","day_of_week","StartCount"]

    # get Ending station counts by year, month, day of week
    df_EndStation = df_ALLTrips.groupby(['end station id','year','month','day_of_week']).size().reset_index(name='EndCount')
    df_EndStation = df_EndStation.sort_values(by=["EndCount"])
    df_EndStation.columns = ["station id","year","month","day_of_week","EndCount"]

    # get counts for each trip by year, month, day of week
    df_TripFrequency = df_ALLTrips.groupby(['start station id','end station id','year','month','day_of_week']).size().reset_index(name='FreqCount')
    df_TripFrequency = df_TripFrequency.sort_values(by=["FreqCount"],ascending=False)

    # get round trip counts, same start and end station by year, month, day of week
    df_Same = df_TripFrequency.loc[df_TripFrequency["start station id"] == df_TripFrequency["end station id"]]
    df_Same = df_Same[["start station id","FreqCount","year","month","day_of_week"]]
    df_Same.columns = ["station id","RTCount","year","month","day_of_week"]

    # merge starting, ending, and round trip data
    df_totals = pd.merge(df_StartStation, df_EndStation, how="outer", on=["station id","year","month","day_of_week"])
    df_totals =pd.merge(df_Same, df_totals,how="outer", on=["station id","year","month","day_of_week"])
    df_totals.fillna(0, inplace=True)

    # get average trip duration for round trip
    df_TripDurationRT = df_ALLTrips.loc[df_ALLTrips["start station id"] == df_ALLTrips["end station id"]]
    df_TripDurationRT = df_TripDurationRT[["tripduration","start station id","year","month","day_of_week"]]

    df_RTDuration = df_TripDurationRT.groupby(["start station id","year","month","day_of_week"]).mean().reset_index()
    df_RTDuration.sort_values("tripduration",ascending=False,inplace=True)
    df_RTDuration.columns = ["station id","year","month","day_of_week","RTAveDuration"]

    # get average trip duration for one way trips
    df_TripDurationOneWay = df_ALLTrips.loc[df_ALLTrips["start station id"] != df_ALLTrips["end station id"]]
    df_TripDurationOneWay = df_TripDurationOneWay[["tripduration","start station id","year","month","day_of_week"]]

    df_OneWayDuration = df_TripDurationOneWay.groupby(["start station id","year","month","day_of_week"]).mean().reset_index()
    df_OneWayDuration.sort_values("tripduration",ascending=False,inplace=True)
    df_OneWayDuration.columns = ["station id","year","month","day_of_week","OneWayAveDuration"]

    # merge durations into totals
    df_totals = pd.merge(df_totals, df_RTDuration, how="outer", on=["station id","year","month","day_of_week"])
    df_totals =pd.merge(df_totals, df_OneWayDuration,how="outer", on=["station id","year","month","day_of_week"])
    df_totals.fillna(0, inplace=True)
    df_TripSummary = df_TripSummary.append(df_totals)

['DataFiles\\JC-202006-citibike-tripdata.csv', 'DataFiles\\NY-202006-citibike-tripdata.csv']
['DataFiles\\JC-202005-citibike-tripdata.csv', 'DataFiles\\NY-202005-citibike-tripdata.csv']
['DataFiles\\JC-202004-citibike-tripdata.csv', 'DataFiles\\NY-202004-citibike-tripdata.csv']
['DataFiles\\JC-202003-citibike-tripdata.csv', 'DataFiles\\NY-202003-citibike-tripdata.csv']
['DataFiles\\JC-202002-citibike-tripdata.csv', 'DataFiles\\NY-202002-citibike-tripdata.csv']
['DataFiles\\JC-202001-citibike-tripdata.csv', 'DataFiles\\NY-202001-citibike-tripdata.csv']
['DataFiles\\JC-201912-citibike-tripdata.csv', 'DataFiles\\NY-201912-citibike-tripdata.csv']
['DataFiles\\JC-201911-citibike-tripdata.csv', 'DataFiles\\NY-201911-citibike-tripdata.csv']
['DataFiles\\JC-201910-citibike-tripdata.csv', 'DataFiles\\NY-201910-citibike-tripdata.csv']
['DataFiles\\JC-201909-citibike-tripdata.csv', 'DataFiles\\NY-201909-citibike-tripdata.csv']
['DataFiles\\JC-201908-citibike-tripdata.csv', 'DataFiles\\NY-201908-c

In [2]:
df_TripSummary.sort_values("station id", inplace=True)
df_TripSummary.to_csv("DataFiles\TripSummary.csv",index=False)
